# Adversarial Search: Playing Dots and Boxes


## Instructions

Total Points: Undegraduates 100, graduate students 110

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play the game Dots and Boxes:

> "Dots and Boxes is a pencil-and-paper game for two players. The game starts with an empty grid of dots. Usually two players take turns adding a single horizontal or vertical line between two unjoined adjacent dots. A player who completes the fourth side of a 1x1 box earns one point and takes another turn. A point is typically recorded by placing a mark that identifies the player in the box, such as an initial. The game ends when no more lines can be placed. The winner is the player with the most points. The board may be of any size grid." (see [Dots and Boxes on Wikipedia](https://en.wikipedia.org/wiki/Dots_and_Boxes))

You can play Dots and Boxes [here](https://www.math.ucla.edu/~tom/Games/dots&boxes.html).

## Task 1: Defining the Search Problem [10 point]

Define the components of the search problem associated with this game:

* Initial state
* Actions
* Transition model
* Test for the terminal state
* Utility for terminal states

### Search problem components for Dots and Boxes
- Initial state: An empty grid of dots of size m x n with no lines drawn; boxes owned mapping is empty; current player is +1 (by convention). The board representation from this notebook is used: a dict with 'size': (rows, cols), 'lines': dict(), 'boxes': dict().
- Actions: Placing a single legal line between two adjacent dots. Represented as (orientation, row, col) where orientation is 'h' or 'v' and (row,col) denotes the starting dot.
- Transition model (result(s,a)): Returns the new board after adding the line a. If adding the line completes one or more 1x1 boxes, those boxes are assigned to the player who made the move and the same player moves again; otherwise the turn switches to the opponent. The board's 'lines' and 'boxes' entries are updated immutably (or copied) to reflect the result.
- Terminal test: The state is terminal when the number of drawn lines equals the total possible number of edges on the grid, i.e., when no legal actions remain (or equivalently when all boxes are claimed).
- Utility for terminal states: The utility is the score difference for the current player or a fixed mapping: utility(board) = (number of boxes owned by player +1) - (number owned by player -1). For a two-player zero-sum game we can return this difference; from the perspective of player +1 positive means win. Tie is 0.

How big is the state space? Give an estimate and explain it.


Estimate: For an (r x c) dots grid there are (r*(c-1)) horizontal edges and ((r-1)*c) vertical edges. Total edges E = r*(c-1) + (r-1)*c = 2rc - r - c. Each edge can be either drawn or not, so an upper bound on the number of possible 'line' configurations is 2^E. In addition, ownership of boxes is determined by which lines are present; but ownership is fully determined by the set of lines, so it doesn't multiply the state space further if we derive boxes from lines.

Therefore the state space size is O(2^E). For a small example: a 5x5 dots grid (which makes a 4x4 box board) has r=c=5, E = 2*5*5 -5 -5 = 40. So the number of possible line configurations is up to 2^40 ≈ 1.1e12. Many of these will be unreachable due to game rules, but this gives an order-of-magnitude upper bound.

How big is the game tree that minimax search will go through? Give an estimate and explain it.


Estimate: The game tree branching factor equals the number of available legal edges at a given state. At the start of an E-edge board the branching factor is E. The depth of the game is E moves (one line per move) but note that completing a box yields an extra move for the same player; this doesn't change the maximum depth but changes which player moves. The total number of leaves (upper bound) is E! (the number of permutations of edges), since a game is an ordering of all edges; this is astronomically large. Using the branching-factor estimate b ≈ E/2 on average and depth d ≈ E, the size is roughly b^d which is exponential in E.

For example: for the 5x5 dots (E=40), the naive upper bound E! ≈ 8.16e47. A branching estimates b≈20 and d≈40 gives 20^40 ≈ 1.1e52. Either way, exhaustive minimax is infeasible for moderate boards; pruning and heuristics (cutoffs) are required.

How big is the game tree that minimax search will go through? Give an estimate and explain it.

Let E be the number of edges (possible lines) on the board. A full game is an ordering of all E edges, so the naive upper bound on the number of leaves is E! (factorial). More usefully, the state-space upper bound is 2^E (each edge drawn or not). In practice the game-tree size is exponential in E and infeasible to search exhaustively for moderate boards; pruning and heuristics are required.


## Task 2: Game Environment and Random Agent [30 point]

You need to think about a data structure to represent the board meaning he placed lines and who finished what box. There are many options. Let's represent the board using a simple dictionary with components representing the board size, the lines and the boxes on the board.

**Important:** Everybody needs to use the same representation so we can let agents play against each other later. 

In [3]:
board = {
    'size': (4, 4),  ### number of rows and columns of dots
    'lines': dict(), ### keys are the set of drawn lines
    'boxes': dict    ### keys are the boxes and the value is the player who completed each box
}

def draw_line(board, orientation, row, col):
    """
    Place a line on an exiting board.
       
    Parameters
    ----------
    board: dict
        the board
    orientation: str
        either 'h' or 'v' for horizontal or vertical
    row, col: int
        index of the starting dot for the line (starting with 0)
    
    """
    
    if orientation not in ['h', 'v']:
        return False
        
    if row < 0 or col < 0:
        return False
        
    if row >= board['size'][0] + (orientation == 'v') or col >= board['size'][1] + (orientation == 'h'):
        return False
        
    if (orientation, row, col) in board['lines']:
        return False
            
    board["lines"][(orientation, row, col)] = True
    return True
    

print(draw_line(board, "h", 1, 1))
print(draw_line(board, "v", 1, 1))

# this should not work
print(draw_line(board, "h", 1, 1))

board

True
True
False


{'size': (4, 4),
 'lines': {('h', 1, 1): True, ('v', 1, 1): True},
 'boxes': dict}

Write code to display the board. **Bonus point: Post your visualization code with an example output to the discussion board. The best visualization will earn you bonus participation points in this class.**

In [18]:
def display_board(board):
    rows, cols = board['size']
    # For dots r x c, there are r rows of dots and c columns of dots. Boxes are (r-1)x(c-1)
    for r in range(rows):
        # horizontal edges and dots
        line = []
        for c in range(cols):
            line.append('.')
            if c < cols-1:
                if ('h', r, c) in board['lines']:
                    line.append('---')
                else:
                    line.append('   ')
        print(''.join(line))
        # vertical edges and box owners (if not last row)
        if r < rows-1:
            line = []
            for c in range(cols):
                if ('v', r, c) in board['lines']:
                    line.append('|')
                else:
                    line.append(' ')
                if c < cols-1:
                    owner = board.get('boxes', {}).get((r, c), None)
                    if owner == 1:
                        line.append(' X ')
                    elif owner == -1:
                        line.append(' O ')
                    else:
                        line.append('   ')
            print(''.join(line))

# Helper functions
def all_edges(size):
    rows, cols = size
    edges = []
    for r in range(rows):
        for c in range(cols-1):
            edges.append(('h', r, c))
    for r in range(rows-1):
        for c in range(cols):
            edges.append(('v', r, c))
    return edges

def actions(board):
    "Return list of available actions (edges not yet drawn)."
    edges = all_edges(board['size'])
    return [e for e in edges if e not in board['lines']]

def _boxes_completed_by_adding(board, action):
    "Return list of box coords completed by adding this action."
    orientation, r, c = action
    rows, cols = board['size']
    completed = []
    # helper to check if a box at (br,bc) is closed
    def box_closed(br, bc, lines):
        needed = set()
        needed.add(('h', br, bc))
        needed.add(('h', br+1, bc))
        needed.add(('v', br, bc))
        needed.add(('v', br, bc+1))
        return all(e in lines for e in needed)

    # consider up to two boxes that share this edge
    if orientation == 'h':
        # top box is at (r-1,c) if r-1 >=0
        if r-1 >= 0 and r-1 < rows-1:
            lines = set(board['lines'].keys()) | {action}
            if box_closed(r-1, c, lines):
                completed.append((r-1, c))
        # bottom box at (r,c) if r < rows-1
        if r < rows-1:
            lines = set(board['lines'].keys()) | {action}
            if box_closed(r, c, lines):
                completed.append((r, c))
    else:
        # vertical edge: left box (r,c-1) and right box (r,c)
        if c-1 >= 0 and c-1 < cols-1:
            lines = set(board['lines'].keys()) | {action}
            if box_closed(r, c-1, lines):
                completed.append((r, c-1))
        if c < cols-1:
            lines = set(board['lines'].keys()) | {action}
            if box_closed(r, c, lines):
                completed.append((r, c))
    return completed

def result(board, action, player):
    # Return a new board after player performs action.
    new = {'size': board['size'], 'lines': dict(board['lines']), 'boxes': dict(board.get('boxes', {}))}
    if action in new['lines']:
        return None, False  # illegal move
    new['lines'][action] = True
    completed = _boxes_completed_by_adding(board, action)
    extra_turn = False
    for b in completed:
        new['boxes'][b] = player
        extra_turn = True
    return new, extra_turn

def terminal(board):
    return len(board['lines']) == len(all_edges(board['size']))

def utility(board):
    # score for player +1 minus player -1
    boxes = board.get('boxes', {})
    s = 0
    for v in boxes.values():
        s += v
    return s

import random
def random_player(board, player=None):
    acts = actions(board)
    if not acts:
        return None
    return random.choice(acts)

# create small board and play one random move
if __name__ == '__main__':
    b = {'size': (4,4), 'lines': dict(), 'boxes': dict()}
    display_board(b)
    a = random_player(b, 1)
    print('Random action chosen:', a)


.   .   .   .
             
.   .   .   .
             
.   .   .   .
             
.   .   .   .
Random action chosen: ('v', 0, 2)


Implement helper functions for:

* The transition model $result(s, a)$.
* The utility function $utility(s)$.
* Check for terminal states $terminal(s)$.
* A check for available actions in each state $actions(s)$.

__Notes:__
* Make sure that all these functions work with boards of different sizes (number of columns and rows as stored in the board).
* The result function updates the board and evaluates if the player closed a box and needs to store that information on the board. Add elements of the form `(row,col): player` to the board dictionary. `row` and `col` are the coordinates for the box and `player` is +1 or -1 representing the player. For example `(0,0): -1` means that the top-left box belongs to the other player. 
* _Important:_ Remember that a player goes again after she completes a box!

In [21]:
def play_game(agent1, agent2, size=(3,3), verbose=False):
    # agents take signature (board, player) -> action. player: +1 or -1
    board = {'size': size, 'lines': dict(), 'boxes': dict()}
    player = 1
    agents = {1: agent1, -1: agent2}
    while not terminal(board):
        act = agents[player](board, player)
        if act is None:
            break
        new_board, extra = result(board, act, player)
        if new_board is None:
            # illegal move: treat as pass
            break
        board = new_board
        if not extra:
            player = -player
    # compute final utility: +1 minus -1 counts
    score = utility(board)
    if score > 0:
        winner = 1
    elif score < 0:
        winner = -1
    else:
        winner = 0
    if verbose:
        display_board(board)
        print('score', score)
    return winner, score, board

def simulate_games(n=200, size=(3,3)):
    results = {1:0, -1:0, 0:0}
    for i in range(n):
        w, s, _ = play_game(random_player, random_player, size=size)
        results[w] += 1
    return results

# Quick run (default 200 games for speed). For 1000 games set n=1000.
if __name__ == '__main__':
    res = simulate_games(200, size=(4,4))
    print('Simulation result (200 random vs random):', res)

Simulation result (200 random vs random): {1: 105, -1: 95, 0: 0}


Implement an agent that plays randomly. Make sure the agent function receives as the percept the board and returns a valid action. Use an agent function definition with the following signature (arguments):

`def random_player(board, player = None): ...`

The argument `player` is used for agents that do not store what side they are playing. The value passed on by the environment should be 1 ot -1 for playerred and yellow, respectively.  See [Experiments section for tic-tac-toe](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_and_or_tree_search.ipynb#Experiments) for an example.

In [24]:
# random_player is implemented earlier
def random_player_wrapper(board, player=None):
    return random_player(board, player)

# Example single action:
if __name__ == '__main__':
    b = {'size': (3,3), 'lines': dict(), 'boxes': dict()}
    print('Random action on empty 3x3 dots board:', random_player(b, 1))

Random action on empty 3x3 dots board: ('v', 1, 0)


Let two random agents play against each other 1000 times. Look at the [Experiments section for tic-tac-toe](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_and_or_tree_search.ipynb#Experiments) to see how the environment uses the agent functions to play against each other.

How often does each player win? Is the result expected?

In [25]:
sizes = (4,4)
n = 1000  
res = simulate_games(n=n, size = sizes)
print(f'Results of {n} random vs random games on a {sizes} dot grid :', res)

Results of 1000 random vs random games on a (4, 4) dot grid : {1: 474, -1: 526, 0: 0}


## Task 3: Minimax Search with Alpha-Beta Pruning [30 points]

### Implement the search starting.

Implement the search starting from a given board and specifying the player and put it into an agent function.
You can use code from the [tic-tac-toe example](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_alpha_beta_tree_search.ipynb).

__Notes:__ 
* Make sure that all your agent functions have a signature consistent with the random agent above.
* The search space for larger board may be too large. You can experiment with smaller boards.
* Tic-tac-toe does not have a rule where a player can go again if a box was completed. You need to adapt the tree search to reflect that rule.

In [16]:
import time

# Minimax with alpha-beta pruning. We must handle the extra-turn rule: when a move completes a box, the same player moves again.
def alphabeta_search(board, player, max_depth=None, eval_fn=None):
    # returns best_action, value, nodes_searched, time_ms
    start = time.time()
    nodes = 0
    def max_value(state, alpha, beta, depth, player_to_move):
        nonlocal nodes
        nodes += 1
        if terminal(state) or (max_depth is not None and depth >= max_depth):
            if eval_fn and not terminal(state):
                return eval_fn(state, player), None
            return utility(state), None
        v = -float('inf')
        best_act = None
        acts = actions(state)
        # simple move ordering: prefer moves that complete boxes
        acts = sorted(acts, key=lambda a: -len(_boxes_completed_by_adding(state, a)))
        for a in acts:
            new_state, extra = result(state, a, player_to_move)
            if new_state is None:
                continue
            if extra:
                val, _ = max_value(new_state, alpha, beta, depth+1, player_to_move)
            else:
                val, _ = min_value(new_state, alpha, beta, depth+1, -player_to_move)
            if val > v:
                v = val
                best_act = a
            if v >= beta:
                return v, best_act
            alpha = max(alpha, v)
        return v, best_act

    def min_value(state, alpha, beta, depth, player_to_move):
        nonlocal nodes
        nodes += 1
        if terminal(state) or (max_depth is not None and depth >= max_depth):
            if eval_fn and not terminal(state):
                return eval_fn(state, player), None
            return utility(state), None
        v = float('inf')
        best_act = None
        acts = actions(state)
        acts = sorted(acts, key=lambda a: -len(_boxes_completed_by_adding(state, a)))
        for a in acts:
            new_state, extra = result(state, a, player_to_move)
            if new_state is None:
                continue
            if extra:
                val, _ = min_value(new_state, alpha, beta, depth+1, player_to_move)
            else:
                val, _ = max_value(new_state, alpha, beta, depth+1, -player_to_move)
            if val < v:
                v = val
                best_act = a
            if v <= alpha:
                return v, best_act
            beta = min(beta, v)
        return v, best_act

    # root: maximize for 'player' (we assume player is to move at root).
    value, action = max_value(board, -float('inf'), float('inf'), 0, player)
    elapsed = (time.time() - start) * 1000.0
    return action, value, nodes, elapsed

# Agent wrapper: keep signature def agent(board, player=None)
def minimax_agent(board, player=None, max_depth=None, eval_fn=None):
    action, value, nodes, ms = alphabeta_search(board, player, max_depth=max_depth, eval_fn=eval_fn)
    return action

# Small tests: run minimax on very small boards to check it chooses box-completing moves when available.
if __name__ == '__main__':
    # create a small 3x3 dots board (2x2 boxes) and set up a state where a move completes a box
    b = {'size': (3,3), 'lines': dict(), 'boxes': dict()}
    # Draw edges so that a single move at ('h',0,0) completes top-left box
    pre_moves = [('h',0,0), ('v',0,0), ('v',0,1)]
    for m in pre_moves:
        b['lines'][m] = True
    display_board(b)
    print('Available actions:', actions(b))
    act = alphabeta_search(b, 1)[0]
    print('Alphabeta chose:', act)

.---.   .
|   |    
.   .   .
         
.   .   .
Available actions: [('h', 0, 1), ('h', 1, 0), ('h', 1, 1), ('h', 2, 0), ('h', 2, 1), ('v', 0, 2), ('v', 1, 0), ('v', 1, 1), ('v', 1, 2)]
Alphabeta chose: ('h', 1, 0)


Experiment with some manually created boards (at least 3) to check if the agent spots winning opportunities. Discuss the results.

In [27]:
# Experiment: test minimax on some manually created boards
def test_manual_boards():
    boards = []
    # 1) simple immediate box available (3x3 dots)
    b1 = {'size': (3,3), 'lines': dict(), 'boxes': dict()}
    for m in [('h',0,0), ('v',0,0), ('v',0,1)]:
        b1['lines'][m] = True
    boards.append(('one-box-available', b1))
    # 2) near endgame small board: many lines filled except a couple
    b2 = {'size': (3,3), 'lines': dict(), 'boxes': dict()}
    # fill almost everything leaving two edges
    for e in all_edges(b2['size']):
        b2['lines'][e] = True
    # remove two edges to create choice
    rem = [('h',0,1), ('v',1,0)]
    for r in rem:
        del b2['lines'][r]
    boards.append(('almost-complete', b2))
    # 3) a medium board with a forced chain start (4x4 dots)
    b3 = {'size': (4,4), 'lines': dict(), 'boxes': dict()}
    # create configuration manually (small chain)
    pre = [('h',0,0), ('h',0,1), ('v',0,0), ('v',0,1), ('v',1,1), ('h',1,0)]
    for m in pre:
        b3['lines'][m] = True
    boards.append(('chain-start', b3))

    results = {}
    for name, board in boards:
        action, value, nodes, ms = alphabeta_search(board, 1, max_depth=6, eval_fn=naive_heuristic)
        results[name] = {'action': action, 'value': value, 'nodes': nodes, 'ms': ms}
    return results

if __name__ == '__main__':
    print('Manual boards test results:')
    print(test_manual_boards())

Manual boards test results:
{'one-box-available': {'action': ('h', 1, 0), 'value': 20, 'nodes': 1115, 'ms': 23.395061492919922}, 'almost-complete': {'action': ('h', 0, 1), 'value': 2, 'nodes': 5, 'ms': 0.0}, 'chain-start': {'action': ('h', 0, 2), 'value': 0, 'nodes': 22485, 'ms': 580.6527137756348}}
{'one-box-available': {'action': ('h', 1, 0), 'value': 20, 'nodes': 1115, 'ms': 23.395061492919922}, 'almost-complete': {'action': ('h', 0, 1), 'value': 2, 'nodes': 5, 'ms': 0.0}, 'chain-start': {'action': ('h', 0, 2), 'value': 0, 'nodes': 22485, 'ms': 580.6527137756348}}


How long does it take to make a move? Start with a smaller board make the board larger. What is the largest board you can solve?

In [28]:
# Timing: measure how long a move takes for different depths/sizes
def measure_move_time(size=(3,3), player=1, depth=2):
    board = {'size': size, 'lines': dict(), 'boxes': dict()}
    action, value, nodes, ms = alphabeta_search(board, player, max_depth=depth, eval_fn=naive_heuristic)
    return {'size': size, 'depth': depth, 'action': action, 'nodes': nodes, 'ms': ms}

def sweep_sizes_depths(sizes=[(3,3),(4,4)], depths=[1,2,3]):
    out = {}
    for s in sizes:
        for d in depths:
            out[(s,d)] = measure_move_time(size=s, depth=d)
    return out

if __name__ == '__main__':
    print('Timing sweep (may be slow for larger depths):')
    print(sweep_sizes_depths(sizes=[(3,3),(4,4)], depths=[1,2]))

Timing sweep (may be slow for larger depths):
{((3, 3), 1): {'size': (3, 3), 'depth': 1, 'action': ('h', 0, 0), 'nodes': 13, 'ms': 0.0}, ((3, 3), 2): {'size': (3, 3), 'depth': 2, 'action': ('h', 0, 0), 'nodes': 35, 'ms': 0.99945068359375}, ((4, 4), 1): {'size': (4, 4), 'depth': 1, 'action': ('h', 0, 0), 'nodes': 25, 'ms': 0.0}, ((4, 4), 2): {'size': (4, 4), 'depth': 2, 'action': ('h', 0, 0), 'nodes': 71, 'ms': 3.0019283294677734}}


### Move ordering

Starting the search with better moves will increase the efficiency of alpha-beta pruning. Describe and implement a simple move ordering strategy. 

Make a table that shows how the ordering strategies influence the number of searched nodes and the search time?

In [50]:
# Move ordering comparison: implement a lightweight alphabeta that can disable ordering
def alphabeta_search_custom(board, player, max_depth=None, eval_fn=None, ordering=True):
    import time
    start = time.time()
    nodes = 0
    def max_value(state, alpha, beta, depth, player_to_move):
        nonlocal nodes
        nodes += 1
        if terminal(state) or (max_depth is not None and depth >= max_depth):
            if eval_fn and not terminal(state):
                return eval_fn(state, player), None
            return utility(state), None
        v = -float('inf')
        best_act = None
        acts = actions(state)
        if ordering:
            acts = sorted(acts, key=lambda a: -len(_boxes_completed_by_adding(state, a)))
        for a in acts:
            new_state, extra = result(state, a, player_to_move)
            if new_state is None:
                continue
            if extra:
                val, _ = max_value(new_state, alpha, beta, depth+1, player_to_move)
            else:
                val, _ = min_value(new_state, alpha, beta, depth+1, -player_to_move)
            if val > v:
                v = val
                best_act = a
            if v >= beta:
                return v, best_act
            alpha = max(alpha, v)
        return v, best_act
    def min_value(state, alpha, beta, depth, player_to_move):
        nonlocal nodes
        nodes += 1
        if terminal(state) or (max_depth is not None and depth >= max_depth):
            if eval_fn and not terminal(state):
                return eval_fn(state, player), None
            return utility(state), None
        v = float('inf')
        best_act = None
        acts = actions(state)
        if ordering:
            acts = sorted(acts, key=lambda a: -len(_boxes_completed_by_adding(state, a)))
        for a in acts:
            new_state, extra = result(state, a, player_to_move)
            if new_state is None:
                continue
            if extra:
                val, _ = min_value(new_state, alpha, beta, depth+1, player_to_move)
            else:
                val, _ = max_value(new_state, alpha, beta, depth+1, -player_to_move)
            if val < v:
                v = val
                best_act = a
            if v <= alpha:
                return v, best_act
            beta = min(beta, v)
        return v, best_act
    value, action = max_value(board, -float('inf'), float('inf'), 0, player)
    elapsed = (time.time() - start) * 1000.0
    return action, value, nodes, elapsed

def compare_move_ordering(board, player=1, depth=3):
    res_ordered = alphabeta_search_custom(board, player, max_depth=depth, eval_fn=naive_heuristic, ordering=True)
    res_unordered = alphabeta_search_custom(board, player, max_depth=depth, eval_fn=naive_heuristic, ordering=False)
    return {'ordered': res_ordered, 'unordered': res_unordered}

if __name__ == '__main__':
    b = {'size': (4,4), 'lines': dict(), 'boxes': dict()}
    print('Compare ordering on empty 4x4 dots (depth=2):')
    print(compare_move_ordering(b, player=1, depth=2))

Compare ordering on empty 4x4 dots (depth=2):
{'ordered': (('h', 0, 0), 0, 71, 0.0), 'unordered': (('h', 0, 0), 0, 71, 0.0)}


### The first few moves

Start with an empty board. This is the worst case scenario for minimax search with alpha-beta pruning since it needs solve all possible games that can be played (minus some pruning) before making the decision. What can you do? 

In [51]:
# Opening policy for the first few moves (simple heuristic)
def opening_policy(board, player=None):
    # Prefer center-ish moves (avoid creating 3-sided boxes for opponent).
    acts = actions(board)
    # compute cost = number of 3-sided boxes created for opponent after playing this move
    def cost_of_move(a):
        new, extra = result(board, a, player if player is not None else 1)
        if new is None:
            return 999
        cnt = 0
        for br in range(board['size'][0]-1):
            for bc in range(board['size'][1]-1):
                if (br,bc) in new.get('boxes', {}):
                    continue
                sides = 0
                for e in [( 'h', br, bc), ('h', br+1, bc), ('v', br, bc), ('v', br, bc+1)]:
                    if e in new['lines']:
                        sides += 1
                if sides == 3:
                    cnt += 1
        return cnt
    scored = sorted(acts, key=lambda a: (cost_of_move(a), abs(a[1]-board['size'][0]/2)+abs(a[2]-board['size'][1]/2)))
    return scored[0] if scored else None

if __name__ == '__main__':
    b = {'size': (5,5), 'lines': dict(), 'boxes': dict()}
    print('Opening policy move on empty 5x5 dots:', opening_policy(b, player=1))

Opening policy move on empty 5x5 dots: ('h', 2, 2)


### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

In [ ]:
# Play minimax (depth-limited) vs random multiple times and report outcomes
def evaluate_minimax_vs_random(n=20, size=(3,3), depth=2):
    results = {1:0, -1:0, 0:0}
    total_nodes = 0
    total_ms = 0.0
    for i in range(n):
        # wrap minimax_agent to include depth
        def mm(board, player=None):
            a, v, nodes, ms = alphabeta_search(board, player if player is not None else 1, max_depth=depth, eval_fn=naive_heuristic)
            nonlocal_total = {'nodes': nodes, 'ms': ms}
            return a
        # To capture nodes/ms we re-run the choice at the root inside play loop; simpler: play with minimax as agent but do not aggregate nodes (we'll measure a single root call)
        w, s, b = play_game(lambda bo, pl: alphabeta_search(bo, pl, max_depth=depth, eval_fn=naive_heuristic)[0], random_player, size=size)
        results[w] += 1
    return results

if __name__ == '__main__':
    print('Minimax(depth=2) vs Random (20 games) on 3x3 dots')
    print(evaluate_minimax_vs_random(n=20, size=(3,3), depth=2))

Minimax(depth=2) vs Random (20 games) on 3x3 dots
{1: 19, -1: 0, 0: 1}


## Task 4: Heuristic Alpha-Beta Tree Search [30 points] 

### Heuristic evaluation function

Define and implement a heuristic evaluation function.

In [26]:
# Heuristic evaluation function: simple features
def naive_heuristic(board, player):
    # Evaluate from perspective of 'player' (+1 or -1). Return higher = better for player.
    # Basic features: (1) current score difference, (2) number of boxes one move away (3 sides drawn) favoring player.
    score = utility(board) * player
    near_boxes = 0
    for br in range(board['size'][0]-1):
        for bc in range(board['size'][1]-1):
            if (br, bc) in board.get('boxes', {}):
                continue
            # count sides drawn around this box
            sides = 0
            for e in [( 'h', br, bc), ('h', br+1, bc), ('v', br, bc), ('v', br, bc+1)]:
                if e in board['lines']:
                    sides += 1
            if sides == 3:
                # if player can move to take it next, it's valuable
                near_boxes += 1
    # heuristic: weighted sum
    return score * 10 + near_boxes * 5

# Depth-limited agent using alphabeta with heuristic cutoff
def heuristic_agent(board, player=None, depth=3):
    # wrapper returning chosen action
    action, val, nodes, ms = alphabeta_search(board, player, max_depth=depth, eval_fn=naive_heuristic)
    return action

# Quick experiment: compare depth=1,2,3 on a small board (measure nodes and time)
def compare_depths(board, player=1, depths=[1,2,3]):
    results = {}
    for d in depths:
        start = time.time()
        action, value, nodes, ms = alphabeta_search(board, player, max_depth=d, eval_fn=naive_heuristic)
        elapsed = (time.time() - start) * 1000.0
        results[d] = {'action': action, 'value': value, 'nodes': nodes, 'ms': elapsed}
    return results

if __name__ == '__main__':
    # pick empty 4x4 dots (3x3 boxes) board and test depths
    b = {'size': (4,4), 'lines': dict(), 'boxes': dict()}
    print('Compare depths on empty board (this may be slow for larger depths)')
    res = compare_depths(b, player=1, depths=[1,2])
    print(res)

Compare depths on empty board (this may be slow for larger depths)
{1: {'action': ('h', 0, 0), 'value': 0, 'nodes': 25, 'ms': 0.9980201721191406}, 2: {'action': ('h', 0, 0), 'value': 0, 'nodes': 71, 'ms': 1.8303394317626953}}


### Cutting off search 

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

In [56]:
# Cutoff experiments: run alphabeta with different cutoff depths and record nodes/time
def cutoff_experiment(board_size=(4,4), depths=[1,2,3], repeats=3):
    summary = {}
    for d in depths:
        nodes_acc = 0
        ms_acc = 0.0
        for _ in range(repeats):
            board = {'size': board_size, 'lines': dict(), 'boxes': dict()}
            a, v, nodes, ms = alphabeta_search(board, 1, max_depth=d, eval_fn=naive_heuristic)
            nodes_acc += nodes
            ms_acc += ms
        summary[d] = {'avg_nodes': nodes_acc/repeats, 'avg_ms': ms_acc/repeats}
    return summary

if __name__ == '__main__':
    print('Cutoff experiment on empty 4x4 dots:')
    print(cutoff_experiment((4,4), depths=[1,2], repeats=2))

Cutoff experiment on empty 4x4 dots:
{1: {'avg_nodes': 25.0, 'avg_ms': 0.0}, 2: {'avg_nodes': 71.0, 'avg_ms': 0.0}}


How many nodes are searched and how long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [57]:
# Node/time sweep by increasing columns (keep rows fixed)
def nodes_time_by_columns(max_cols=5, rows=4, depth=2):
    results = {}
    for c in range(2, max_cols+1):
        size = (rows, c)
        board = {'size': size, 'lines': dict(), 'boxes': dict()}
        a, v, nodes, ms = alphabeta_search(board, 1, max_depth=depth, eval_fn=naive_heuristic)
        results[size] = {'nodes': nodes, 'ms': ms}
    return results

if __name__ == '__main__':
    print('Nodes/time by columns (rows=4, depth=2):')
    print(nodes_time_by_columns(max_cols=5, rows=4, depth=2))

Nodes/time by columns (rows=4, depth=2):
{(4, 2): {'nodes': 29, 'ms': 0.48732757568359375}, (4, 3): {'nodes': 50, 'ms': 0.3674030303955078}, (4, 4): {'nodes': 71, 'ms': 0.0}, (4, 5): {'nodes': 92, 'ms': 6.028652191162109}}


### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [58]:
# Heuristic vs Heuristic: play two heuristic agents against each other (different depths)
def play_heuristics(depth_a=1, depth_b=2, size=(4,4)):
    def agent_a(board, player=None):
        return alphabeta_search(board, player, max_depth=depth_a, eval_fn=naive_heuristic)[0]
    def agent_b(board, player=None):
        return alphabeta_search(board, player, max_depth=depth_b, eval_fn=naive_heuristic)[0]
    winner, score, final = play_game(agent_a, agent_b, size=size)
    return {'winner': winner, 'score': score}

if __name__ == '__main__':
    print('Heuristic depth 1 vs depth 2 on 4x4 dots:')
    print(play_heuristics(1,2,(4,4)))

Heuristic depth 1 vs depth 2 on 4x4 dots:
{'winner': -1, 'score': -9}


## Tournament task [+1 to 5% bonus on your course grade; will be assigned separately]

Find another student and let your best agent play against the other student's best player. You are allowed to use any improvements you like as long as you code it yourself. We will set up a class tournament on Canvas. This tournament will continue after the submission deadline.

## Graduate student advanced task: Pure Monte Carlo Search and Best First Move [10 point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+5 Bonus point].

### Pure Monte Carlo Search

Implement Pure Monte Carlo Search (see [tic-tac-toe-example](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_pure_monte_carlo_search.ipynb)) and investigate how this search performs on the test boards that you have used above. 

In [ ]:
# Your code/ answer goes here.

### Best First Move

How would you determine what the best first move for a standard board ($5 \times 5$) is? You can use Pure Monte Carlo Search or any algorithms that you have implemented above.

In [20]:
# Your code/ answer goes here.